<a href="https://colab.research.google.com/github/FoxPowerGH/GenAI-Journey/blob/main/Multi_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Optimización de la Interacción con  un LLM**

**Gestión de Solicitudes Múltiples**


Tradicionalmente, la generación de código a través de modelos de lenguaje (LLM) se ha basado en el envío de un único mensaje que produce un resultado inmediato. No obstante, una estrategia más efectiva y flexible consiste en adoptar un enfoque conversacional, en el cual el usuario puede interactuar con el modelo para solicitar modificaciones y optimizar la precisión de la respuesta.

Este enfoque conversacional permite ajustar gradualmente los resultados, facilitando la producción de soluciones más alineadas con los requerimientos específicos. Sin embargo, en ciertos casos, puede ser ventajoso consolidar la interacción en un único mensaje final que sintetice la conversación previa. Este método no solo mejora la claridad y coherencia del código resultante, sino que también contribuye a su mantenibilidad, al reducir la dependencia de múltiples interacciones intermedias.

La elección entre un proceso iterativo o un mensaje unificado debe evaluarse en función de la complejidad del problema y los objetivos del desarrollo, priorizando la eficiencia y la calidad del código generado.


## **Interacción Conversacional con un LLM: Optimización y Mantenibilidad**



En este módulo, exploraremos cómo aprovechar esta interacción dinámica para solicitar modificaciones y optimizar la calidad de las respuestas producidas por el LLM. Asimismo, analizaremos la posibilidad de consolidar la conversación en un único mensaje final que sintetice el intercambio previo. Este enfoque no solo contribuye a la coherencia del resultado, sino que también facilita la mantenibilidad del código, al reducir la fragmentación del proceso de generación de respuestas.

## **Configuración del Ambiente - Google CoLab**

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# GoogleAI Secrets
if COLAB:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    os.environ["SYSTEM_INSTRUCTION"] = userdata.get('SYSTEM_INSTRUCTION')

# Instalo las librerías requeridas en CoLab
if COLAB:
    #!pip install langchain langchain_openai
    !pip install -U  langchain-google-genai
    !pip install -U -q "google-generativeai>=0.8.4"

Note: using Google CoLab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.3 MB/s eta 0:00:00


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## **Veamos el resultado de esta consulta a nuestro LLM**

In [33]:
# Definimos las librerías con las que vamos a trabajar
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts.chat import PromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage
from IPython.display import display_markdown

TEMPLATE = """La siguiente es una conversación amigable entre un humano y una IA para generar
código en Python. Si tienes notas sobre el código, colócalas antes del código. Cualquier nota
sobre la ejecución debe ir después del código. Si mezclas notas con el código, conviértelas
en comentarios.
Asegúrate de agregar comentarios adecuados dentro del código, ordenar los imports y seguir el formato PEP-8.

Current conversation:
{history}
Human: {input}
Code Assistant:"""

PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)


def start_conversation():
    # Realizamos la consulta al modelo LLM con la API key
    llm =  ChatGoogleGenerativeAI(
      model="gemini-1.5-flash",
      temperature= 0.0,
      max_tokens= 1024)

    # Inicializamos memory and conversation
    memory = ConversationBufferWindowMemory()
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def generate_code(conversation, prompt):
    print("Model response:")
    output = conversation.invoke(prompt)
    display_markdown(output['response'], raw=True)

In [34]:
# Respuesta del Modelo LLM
conversation = start_conversation()
generate_code(conversation, """Escribir un programa en Python,una función para calcular el interes de un plazo fijo.  Capital inicial, años e interes enviar como parametros.""")


Model response:


Notas:

La función `calcular_interes` calcula el interés simple ganado en un plazo fijo.  Asume un interés anual simple y no considera intereses compuestos.  Se incluyen validaciones para asegurar que los parámetros de entrada sean válidos (números positivos).  Se manejan las excepciones para entradas inválidas.


```python
import sys

def calcular_interes(capital_inicial, años, interes):
    """
    Calcula el interés simple ganado en un plazo fijo.

    Args:
        capital_inicial: El capital inicial invertido (debe ser un número positivo).
        años: El número de años de la inversión (debe ser un número positivo).
        interes: La tasa de interés anual (debe ser un número positivo).

    Returns:
        El interés ganado, o None si los parámetros de entrada son inválidos.
    """
    # Validación de parámetros de entrada
    if not all(isinstance(arg, (int, float)) and arg > 0 for arg in [capital_inicial, años, interes]):
        print("Error: El capital inicial, los años y la tasa de interés deben ser números positivos.")
        return None

    # Cálculo del interés simple
    interes_ganado = capital_inicial * años * (interes / 100)  # Se divide por 100 para convertir el porcentaje a decimal.
    return interes_ganado


if __name__ == "__main__":
    try:
        capital_inicial = float(input("Ingrese el capital inicial: "))
        años = float(input("Ingrese el número de años: "))
        interes = float(input("Ingrese la tasa de interés anual (%): "))

        interes_total = calcular_interes(capital_inicial, años, interes)

        if interes_total is not None:
            print(f"El interés ganado después de {años} años es: {interes_total:.2f}")

    except ValueError:
        print("Error: Ingrese valores numéricos válidos.")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        sys.exit(1) # Salir con código de error para indicar fallo

```

Notas sobre la ejecución:

El programa solicita al usuario que ingrese el capital inicial, el número de años y la tasa de interés.  Luego, llama a la función `calcular_interes` para calcular el interés ganado.  El resultado se imprime en la consola, formateado a dos decimales.  Se incluye manejo de excepciones para entradas inválidas (no numéricas) y otros errores inesperados.  El programa termina con un código de error si ocurre una excepción no manejada.  Para ejecutar el código, guárdalo en un archivo (por ejemplo, `interes.py`) y ejecútalo desde la línea de comandos usando `python interes.py`.

## **Solicitamos un cambio al código generado**



In [35]:
generate_code(conversation, """Podría actualizar el programa para utilizar interes compuesto.""")

Model response:


Notas:

La función `calcular_interes_compuesto` calcula el interés compuesto ganado en un plazo fijo.  Utiliza la fórmula del interés compuesto:  `capital_final = capital_inicial * (1 + interes/100)**años`.  Se incluyen validaciones para asegurar que los parámetros de entrada sean válidos (números positivos). Se manejan las excepciones para entradas inválidas.  La función retorna el interés ganado (diferencia entre capital final y capital inicial).


```python
import sys

def calcular_interes_compuesto(capital_inicial, años, interes):
    """
    Calcula el interés compuesto ganado en un plazo fijo.

    Args:
        capital_inicial: El capital inicial invertido (debe ser un número positivo).
        años: El número de años de la inversión (debe ser un número positivo).
        interes: La tasa de interés anual (debe ser un número positivo).

    Returns:
        El interés ganado, o None si los parámetros de entrada son inválidos.
    """
    # Validación de parámetros de entrada
    if not all(isinstance(arg, (int, float)) and arg > 0 for arg in [capital_inicial, años, interes]):
        print("Error: El capital inicial, los años y la tasa de interés deben ser números positivos.")
        return None

    # Cálculo del interés compuesto
    capital_final = capital_inicial * (1 + interes / 100) ** años
    interes_ganado = capital_final - capital_inicial
    return interes_ganado


if __name__ == "__main__":
    try:
        capital_inicial = float(input("Ingrese el capital inicial: "))
        años = float(input("Ingrese el número de años: "))
        interes = float(input("Ingrese la tasa de interés anual (%): "))

        interes_total = calcular_interes_compuesto(capital_inicial, años, interes)

        if interes_total is not None:
            print(f"El interés ganado después de {años} años es: {interes_total:.2f}")

    except ValueError:
        print("Error: Ingrese valores numéricos válidos.")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        sys.exit(1)

```

Notas sobre la ejecución:

El programa solicita al usuario que ingrese el capital inicial, el número de años y la tasa de interés. Luego, llama a la función `calcular_interes_compuesto` para calcular el interés compuesto ganado. El resultado se imprime en la consola, formateado a dos decimales. Se incluye manejo de excepciones para entradas inválidas (no numéricas) y otros errores inesperados. El programa termina con un código de error si ocurre una excepción no manejada. Para ejecutar el código, guárdalo en un archivo (por ejemplo, `interes_compuesto.py`) y ejecútalo desde la línea de comandos usando `python interes_compuesto.py`.

## **Solictamos un segundo cambio al código generado**

In [36]:
generate_code(conversation, """Podría actualizar el programa para que considere aportaciones mensuales.""")

Model response:


Notas:

La función `calcular_interes_compuesto_con_aportaciones` calcula el interés compuesto ganado en un plazo fijo considerando aportaciones mensuales adicionales.  Utiliza una iteración para calcular el interés acumulado mes a mes. Se incluyen validaciones para asegurar que los parámetros de entrada sean válidos (números positivos). Se manejan las excepciones para entradas inválidas. La función retorna el interés ganado (diferencia entre capital final y capital inicial + aportaciones totales).


```python
import sys

def calcular_interes_compuesto_con_aportaciones(capital_inicial, años, interes, aportacion_mensual):
    """
    Calcula el interés compuesto ganado en un plazo fijo con aportaciones mensuales.

    Args:
        capital_inicial: El capital inicial invertido (debe ser un número positivo).
        años: El número de años de la inversión (debe ser un número positivo).
        interes: La tasa de interés anual (debe ser un número positivo).
        aportacion_mensual: La aportación mensual adicional (debe ser un número positivo o cero).

    Returns:
        El interés ganado, o None si los parámetros de entrada son inválidos.
    """
    # Validación de parámetros de entrada
    if not all(isinstance(arg, (int, float)) and arg >= 0 for arg in [capital_inicial, años, interes, aportacion_mensual]):
        print("Error: El capital inicial, los años, la tasa de interés y la aportación mensual deben ser números no negativos.")
        return None

    # Cálculo del interés compuesto con aportaciones mensuales
    meses = años * 12
    capital_actual = capital_inicial
    aportaciones_totales = 0

    for _ in range(meses):
        capital_actual += aportacion_mensual
        aportaciones_totales += aportacion_mensual
        capital_actual *= (1 + interes / (100 * 12)) # Interés mensual

    interes_ganado = capital_actual - capital_inicial - aportaciones_totales
    return interes_ganado


if __name__ == "__main__":
    try:
        capital_inicial = float(input("Ingrese el capital inicial: "))
        años = float(input("Ingrese el número de años: "))
        interes = float(input("Ingrese la tasa de interés anual (%): "))
        aportacion_mensual = float(input("Ingrese la aportación mensual: "))

        interes_total = calcular_interes_compuesto_con_aportaciones(capital_inicial, años, interes, aportacion_mensual)

        if interes_total is not None:
            print(f"El interés ganado después de {años} años es: {interes_total:.2f}")

    except ValueError:
        print("Error: Ingrese valores numéricos válidos.")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        sys.exit(1)

```

Notas sobre la ejecución:

El programa solicita al usuario que ingrese el capital inicial, el número de años, la tasa de interés y la aportación mensual. Luego, llama a la función `calcular_interes_compuesto_con_aportaciones` para calcular el interés compuesto ganado considerando las aportaciones mensuales. El resultado se imprime en la consola, formateado a dos decimales. Se incluye manejo de excepciones para entradas inválidas (no numéricas) y otros errores inesperados. El programa termina con un código de error si ocurre una excepción no manejada. Para ejecutar el código, guárdalo en un archivo (por ejemplo, `interes_compuesto_aportaciones.py`) y ejecútalo desde la línea de comandos usando `python interes_compuesto_aportaciones.py`.

## **Probamos el programa generado por el LLM**

In [41]:
import sys

def calcular_interes_compuesto_con_aportaciones(capital_inicial, años, interes, aportacion_mensual):
    """
    Calcula el interés compuesto ganado en un plazo fijo con aportaciones mensuales.

    Args:
        capital_inicial: El capital inicial invertido (debe ser un número positivo).
        años: El número de años de la inversión (debe ser un número positivo).
        interes: La tasa de interés anual (debe ser un número positivo).
        aportacion_mensual: La aportación mensual adicional (debe ser un número positivo o cero).

    Returns:
        El interés ganado, o None si los parámetros de entrada son inválidos.
    """
    # Validación de parámetros de entrada
    if not all(isinstance(arg, (int, float)) and arg >= 0 for arg in [capital_inicial, años, interes, aportacion_mensual]):
        print("Error: El capital inicial, los años, la tasa de interés y la aportación mensual deben ser números no negativos.")
        return None

    # Cálculo del interés compuesto con aportaciones mensuales
    meses = int(años * 12)
    capital_actual = capital_inicial
    aportaciones_totales = 0

    for _ in range(meses):
        capital_actual += aportacion_mensual
        aportaciones_totales += aportacion_mensual
        capital_actual *= (1 + interes / (100 * 12)) # Interés mensual

    interes_ganado = capital_actual - capital_inicial - aportaciones_totales
    return interes_ganado


if __name__ == "__main__":
    try:
        capital_inicial = float(input("Ingrese el capital inicial: "))
        años = float(input("Ingrese el número de años: "))
        interes = float(input("Ingrese la tasa de interés anual (%): "))
        aportacion_mensual = float(input("Ingrese la aportación mensual: "))

        interes_total = calcular_interes_compuesto_con_aportaciones(capital_inicial, años, interes, aportacion_mensual)

        if interes_total is not None:
            print(f"El interés ganado después de {años} años es: {interes_total:.2f}")

    except ValueError:
        print("Error: Ingrese valores numéricos válidos.")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        sys.exit(1)

Ingrese el capital inicial: 10000
Ingrese el número de años: 14
Ingrese la tasa de interés anual (%): 8
Ingrese la aportación mensual: 300
El interés ganado después de 14.0 años es: 63157.66
